In [1]:
import os,sys,time
from collections import OrderedDict
from aem_client import *
import schedule
import time

In [2]:
domain = 'gain-insights'
username = 'Unnathikr'
password = 'moon@2000UNNAT'
aem_machine_name = '7pdxcq2'


In [3]:
import psycopg2
conn = psycopg2.connect(
   database="postgres", user='postgres', password='moon@2000U', host='localhost', port= '5432')
cursor = conn.cursor()
conn.autocommit = True

In [4]:
server = 'New Server'
task = 'kafka'
schema ='DES'
table = '{}'


In [5]:
def automated_source_counts():
    domain_username = '{0}\\{1}'.format(domain,username)
    username_password_str = str.encode('{0}:{1}'.format(domain_username,password))
    b64_username_password = base64.b64encode(username_password_str).decode('ascii')
    aem_client= AemClient(b64_username_password,aem_machine_name,port=443,url="https://7pdxcq2/attunityenterprisemanager",
                      verify_certificate = False,authentication_method=AuthenticationMethod.ACTIVE_DIRECTORY)
    
    res1 = aem_client.get_table_list(server,task,schema='',table='')

    for i in range(len(res1.tablelist)):
            res0= aem_client.get_table_statuses(server,task,schema='',table='{}'.format(res1.tablelist[i].table))
            table='{}'.format(res1.tablelist[i].table)
            schema='{}'.format(res1.tablelist[i].schema)
            for t in res0.table_details:
                # l=table,t.table_cdc_info.insert_count,t.table_cdc_info.update_count,t.table_cdc_info.delete_count
                messageinserts=t.table_cdc_info.insert_count
                messageupdates=t.table_cdc_info.update_count
                messagedeletes=t.table_cdc_info.delete_count
                activitydate=t.table_cdc_info.last_update_time
                activitydatesplit = activitydate.split("T")
            update_query="UPDATE dataprocesslog SET message_insert_count = %s,message_update_count = %s,message_delete_count = %s where sourcetablename = %s and activity_date= %s"
            tablename=schema+'.'+table
            cursor.execute(update_query,(messageinserts,messageupdates,messagedeletes,tablename,activitydatesplit[0]))
    
    



In [6]:
schedule.every(4).hours.do(automated_source_counts)
while True:
    schedule.run_pending()
    time.sleep(1)

KeyboardInterrupt: 